In [13]:
import tensorflow as tf
import tensorflow_hub as hub

import tensorflow_datasets as tfds

import time

from PIL import Image
import requests
from io import BytesIO

import matplotlib.pyplot as plt
import numpy as np

import os
import pathlib

In [14]:
#Construct imagenet logit-to-class-name dictionary (imagenet_int_to_str)
!wget https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt

imagenet_int_to_str = {}

with open('ilsvrc2012_wordnet_lemmas.txt', 'r') as f:
  for i in range(1000):
    row = f.readline()
    row = row.rstrip()
    imagenet_int_to_str.update({i: row})

--2020-11-29 21:08:31--  https://storage.googleapis.com/bit_models/ilsvrc2012_wordnet_lemmas.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.204.128, 172.217.203.128, 142.250.98.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.204.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21675 (21K) [text/plain]
Saving to: ‘ilsvrc2012_wordnet_lemmas.txt.1’

ilsvrc2012_wordnet_ 100%[===================>]  21.17K  --.-KB/s    in 0s      

2020-11-29 21:08:31 (111 MB/s) - ‘ilsvrc2012_wordnet_lemmas.txt.1’ saved [21675/21675]



In [15]:
# Load feature vector extractor into KerasLayer
r50x1_url = "https://tfhub.dev/google/bit/m-r50x1/1"
feat_vec_layer = hub.KerasLayer(r50x1_url)

In [16]:
def preprocess_image(image):
  image = np.array(image)
  # reshape into shape [batch_size, height, width, num_channels]
  img_reshaped = tf.reshape(image, [1, image.shape[0], image.shape[1], image.shape[2]])
  # Use `convert_image_dtype` to convert to floats in the [0,1] range.
  image = tf.image.convert_image_dtype(img_reshaped, tf.float32)  
  return image

def load_image_from_url(url):
  """Returns an image with shape [1, height, width, num_channels]."""
  response = requests.get(url)
  image = Image.open(BytesIO(response.content))
  image = preprocess_image(image)
  return image

In [17]:
# test feature vector

# Load image
img_url = "https://p0.pikrepo.com/preview/853/907/close-up-photo-of-gray-elephant.jpg"
image = load_image_from_url(img_url)

# Run model on image
logits = feat_vec_layer(image)

In [18]:
logits # 2048-dimensional feature vector

<tf.Tensor: shape=(1, 2048), dtype=float32, numpy=
array([[0.31866226, 0.        , 8.563895  , ..., 0.79412353, 0.5398085 ,
        6.881541  ]], dtype=float32)>

In [19]:
class RGB_TL_model(tf.keras.Model):
  """transfer learning model using feature vector and custom new head"""

  def __init__(self, num_classes, embedding_layer):
    super().__init__()

    self.num_classes = num_classes
    self.head = tf.keras.layers.Dense(num_classes, kernel_initializer='zeros')
    self.embedding_layer = embedding_layer
  
  def call(self, images):
    img_embedding = self.embedding_layer(images)
    pred = self.head(img_embedding)
    
    return pred

In [20]:
NUM_CLASSES = 2

model = RGB_TL_model(num_classes=NUM_CLASSES, embedding_layer=feat_vec_layer)

In [26]:
IM_SIZE = (640, 480)

model.build(input_shape=(None, 640,480,3))

In [27]:
model.summary()

Model: "rgb_tl_model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              multiple                  4098      
_________________________________________________________________
keras_layer_1 (KerasLayer)   multiple                  23500352  
Total params: 23,504,450
Trainable params: 4,098
Non-trainable params: 23,500,352
_________________________________________________________________


In [28]:
model(image)

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[0., 0.]], dtype=float32)>

In [ ]:
IM_SIZE = (640, 480)
rgb_dataset = tf.keras.preprocessing.image_dataset_from_directory(rgb_out_path, labels='inferred', label_mode='categorical', 
                                                                  image_size=IM_SIZE, batch_size=32, shuffle=True)

In [ ]:
model.fit(rgb_dataset, epochs=1)